# Preprocessing: Water treatment plants

**Objective**: Prepare data from the **MERKUR** dataset for use in machine learning algorithms.

**Background**: The MERKUR project, based in the *Research Centre for Built Environment, Climate, Water Technology and Digitalisation* at VIA University College, collects and analyzes data from water treatment plants in Denmark. In short, the project aims to understand how water treatment plants are run, and the results are then ideally used to optimize the running of water treatment plants. However, the dataset is, as of now, relatively "dirty" in a machine learning context: There are many missing values, outliers, a mix of categorical and numeric data, etc.

**Data Source**: The dataset has kindly been provided to us by Senior Associate Professor Loren Mark Ramsay. You can read more [here](https://en.via.dk/research/built-environment-climate-water-technology-and-digitalisation/water-treatment-and-distribution) and [here](https://www.ucviden.dk/en/projects/merkur-national-web-baseret-dataplatform-til-drikkevandsbehandlin).

Note that we are only working with a subset of the full database. This subset is saved as an Excel file, `merkur.xlsx`.

#### Overall Instructions
1. Explore the dataset to understand the features and their distributions.
2. Preprocess the data, handling any missing values, outliers, etc.

Below some suggestions are given but the assignment is relatively "free".

Best of luck with your analysis!

In [1]:
import pandas as pd

# Assuming you have already read the Excel file into a DataFrame
raw_data = pd.read_excel(r"merkur.xlsx")

# - Overviwe of missing values in the dataset
missing_values = raw_data.isnull().sum()

# - Percentage of missing values in the dataset
missing_percentage = (missing_values / len(raw_data)) * 100

# * Create a new dataframe with the missing values
missing_summary = pd.DataFrame({'Missing Values': missing_values, 'Percentage': missing_percentage, 'Data Type': raw_data.dtypes})
missing_summary = missing_summary.sort_values(by='Percentage', ascending=False)
display(missing_summary)

,Missing Values,Percentage,Data Type
UniformityCoefficient,51,64.556962,float64
SumOfld_layer,39,49.367089,float64
TotalEBCT,39,49.367089,float64
AverageFilterBedVolume,34,43.037975,float64
OverallFilterGrainSizeMin,34,43.037975,float64
OverallFilterGrainSizeMax,34,43.037975,float64
AverageTotalFilterDepth,33,41.772152,float64
TankExploitation,30,37.974684,float64
HLR_BW,27,34.177215,float64
Stage1HLR,21,26.582278,float64


In [2]:
# ! Extract the columns with more than 40% missing values
columns_to_drop = missing_summary[missing_summary['Percentage'] > 40]

data_cleaned = raw_data.copy()
data_cleaned = data_cleaned.drop(columns=columns_to_drop.index)
data_cleaned = data_cleaned.drop(columns=['WaterworksName'])

# * Impute missin g values for numerical columns using the median
numerical_columns = data_cleaned.select_dtypes(include=['float64','int64']).columns
data_cleaned[numerical_columns] = data_cleaned[numerical_columns].fillna(data_cleaned[numerical_columns].median())

# * Impute missing values for categorical columns using the mode
categorical_columns = data_cleaned.select_dtypes(include=['object']).columns

# Fill missing data with mdoe value of current col
for column in categorical_columns:
    mode_value = data_cleaned[column].mode()[0]
    data_cleaned[column] = data_cleaned[column].fillna(mode_value)
    
display(data_cleaned)

,TotalFilters,MaxTypicalFlow,AverageFilterArea,AverageTypicalRunVolume,AverageBackwashVolume,PrimaryTrigger,FilterExploitation,AerationType,OxygenFactor,Stage1HLR,Footprint,UFRV,BW%,HLR_BW,TankCapacity,TankExploitation,GravityPressureMixed,Stages,AbstractedVolume
0,6,50.00,6.000000,5200.00000,10.00,Time,57.886530,Cascade,1.880000,1.388889,143.341204,866.666667,0.414525,30.889453,27.294778,19.402985,Gravity,Single,253543
1,4,110.00,4.908739,300.00000,28.00,Volume,63.521586,Air injection,1.620000,11.204508,16.714412,40.743665,23.000062,30.557749,2.576075,19.402985,Pressure,Double,612094
2,4,120.00,15.343900,3800.00000,83.30,Volume,64.908486,Bottom aeration,2.213333,3.935020,44.693822,249.217919,2.192086,32.791831,21.825600,19.402985,Gravity,Double,682318
3,2,120.00,16.000000,1200.00000,26.00,Volume,63.279585,Cascade,2.073333,3.742502,194.588021,75.000000,2.166907,30.889453,12.554300,19.402985,Gravity,Single,156998
4,2,120.00,1.495000,250.00000,4.50,Volume,63.279585,Cascade,1.313333,3.742502,412.300024,167.224093,1.818182,30.889453,52.220566,19.402985,Gravity,Single,6710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,16,1426.00,47.940002,3250.00000,110.00,Mix,91.698648,Cascade,1.546667,5.242647,24.106636,258.637091,0.178441,30.889453,9.176959,39.682540,Gravity,Double,11454775
75,20,1407.29,10.178760,4500.00000,57.00,Volume,89.405620,Passive plate aerator,2.133333,6.912876,22.050800,442.097088,1.236822,51.086775,6.040393,19.402985,Pressure,Single,11021800
76,6,250.00,14.752500,2331.50137,51.57,Time,19.331598,Bottom aeration,1.673333,2.824380,209.076387,158.041107,4.446077,30.367735,22.367525,45.833333,Gravity,Single,423362
77,12,120.00,37.437401,4400.00000,134.00,Volume,63.279585,Cascade,2.200000,3.742502,152.513970,93.489396,35.386015,32.053507,15.097670,19.402985,Gravity,Double,1521895


In [3]:
data_cleaned.isna().sum()

TotalFilters               0
MaxTypicalFlow             0
AverageFilterArea          0
AverageTypicalRunVolume    0
AverageBackwashVolume      0
PrimaryTrigger             0
FilterExploitation         0
AerationType               0
OxygenFactor               0
Stage1HLR                  0
Footprint                  0
UFRV                       0
BW%                        0
HLR_BW                     0
TankCapacity               0
TankExploitation           0
GravityPressureMixed       0
Stages                     0
AbstractedVolume           0
dtype: int64

## __Fill missing variables:__

### *Numerical Values:*
We fill all missing numerical values with the median of the current column values. This is partly because using the median value is resistant to outliers which helps the preservation of the central tendancy.

### *Categorical Values:*
We fill all missing categorical values with the mode of the current column. The reason we do this is because even categorical values can be filled using the mode value of a speciffic column, since it is the most common value present in the column. Another reason is the preservation of the distribution within the dataset. Imputing using the mode variable helps maintain the distribution of the categorical variable.

In [4]:
#import pandas as pd
#from scipy import stats
#import numpy as np
#
#irrelevant_columns = ['TotalFilters', 'TankCapacity', 'OxygenFactor']
#relevant_columns = [col for col in numerical_columns if col not in irrelevant_columns]
#
## Calculate the z-score for all values in relevant columns
#for col in relevant_columns:
#    z_scores = stats.zscore(data_cleaned[col])
#    abs_z_scores = np.abs(z_scores)
#    filtered_entries = (abs_z_scores < 3)  # Adjust 3 as per requirement
#    data_cleaned = data_cleaned[filtered_entries]
#
## Reset index
#data_cleaned.reset_index(drop=True, inplace=True)
#
## Now, data_cleaned has only the filtered entries with a new continuous index
#display(len(data_cleaned))

In [5]:
from sklearn.preprocessing import MinMaxScaler

numerical_columns = data_cleaned.select_dtypes(include=['float64','int64']).columns

# Declare 
scaler = MinMaxScaler()
scaled = scaler.fit_transform(data_cleaned[numerical_columns])
scaled_df = pd.DataFrame(scaled, columns=numerical_columns)

display(scaled_df)

,TotalFilters,MaxTypicalFlow,AverageFilterArea,AverageTypicalRunVolume,AverageBackwashVolume,FilterExploitation,OxygenFactor,Stage1HLR,Footprint,UFRV,BW%,HLR_BW,TankCapacity,TankExploitation,AbstractedVolume
0,0.263158,0.030645,0.100245,0.078884,0.054519,0.416917,0.610390,0.038377,0.306610,0.758179,0.009651,0.450317,0.263365,0.202788,0.021561
1,0.157895,0.072913,0.076834,0.000797,0.161185,0.472130,0.399351,0.686409,0.018993,0.000000,0.649245,0.445406,0.000000,0.202788,0.052881
2,0.157895,0.079958,0.300703,0.056574,0.488889,0.485718,0.880952,0.206473,0.082545,0.191375,0.059990,0.478478,0.205094,0.202788,0.059015
3,0.052632,0.079958,0.314779,0.015139,0.149333,0.469758,0.767316,0.193763,0.423011,0.031447,0.059277,0.450317,0.106313,0.202788,0.013128
4,0.052632,0.079958,0.003597,0.000000,0.021926,0.469758,0.150433,0.193763,0.917517,0.116106,0.049401,0.450317,0.528936,0.202788,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,0.789474,1.000000,1.000000,0.047809,0.647111,0.748210,0.339827,0.292804,0.035784,0.200021,0.002966,0.450317,0.070329,0.445497,1.000000
75,1.000000,0.986819,0.189893,0.067729,0.333037,0.725743,0.816017,0.403073,0.031114,0.368434,0.032938,0.749303,0.036911,0.202788,0.962179
76,0.263158,0.171539,0.288016,0.033171,0.300859,0.039153,0.442641,0.133149,0.455919,0.107676,0.123820,0.442593,0.210868,0.519111,0.036395
77,0.578947,0.079958,0.774684,0.066135,0.789333,0.469758,0.870130,0.193763,0.327445,0.048419,1.000000,0.467548,0.133411,0.202788,0.132353


In [6]:
# combine the scaled dataframe with the original categorical columns
combined_df = pd.concat([scaled_df, data_cleaned[categorical_columns]], axis=1)

# One hot encode the combined dataframe
one_hot_encoded_df = pd.get_dummies(combined_df, columns=categorical_columns)

display(one_hot_encoded_df)

,TotalFilters,MaxTypicalFlow,AverageFilterArea,AverageTypicalRunVolume,AverageBackwashVolume,FilterExploitation,OxygenFactor,Stage1HLR,Footprint,UFRV,...,AerationType_Other,AerationType_Passive plate aerator,AerationType_Pure oxygen injection,GravityPressureMixed_Gravity,GravityPressureMixed_Mixed,GravityPressureMixed_Pressure,Stages_Double,Stages_Mixed,Stages_Single,Stages_Triple
0,0.263158,0.030645,0.100245,0.078884,0.054519,0.416917,0.610390,0.038377,0.306610,0.758179,...,False,False,False,True,False,False,False,False,True,False
1,0.157895,0.072913,0.076834,0.000797,0.161185,0.472130,0.399351,0.686409,0.018993,0.000000,...,False,False,False,False,False,True,True,False,False,False
2,0.157895,0.079958,0.300703,0.056574,0.488889,0.485718,0.880952,0.206473,0.082545,0.191375,...,False,False,False,True,False,False,True,False,False,False
3,0.052632,0.079958,0.314779,0.015139,0.149333,0.469758,0.767316,0.193763,0.423011,0.031447,...,False,False,False,True,False,False,False,False,True,False
4,0.052632,0.079958,0.003597,0.000000,0.021926,0.469758,0.150433,0.193763,0.917517,0.116106,...,False,False,False,True,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,0.789474,1.000000,1.000000,0.047809,0.647111,0.748210,0.339827,0.292804,0.035784,0.200021,...,False,False,False,True,False,False,True,False,False,False
75,1.000000,0.986819,0.189893,0.067729,0.333037,0.725743,0.816017,0.403073,0.031114,0.368434,...,False,True,False,False,False,True,False,False,True,False
76,0.263158,0.171539,0.288016,0.033171,0.300859,0.039153,0.442641,0.133149,0.455919,0.107676,...,False,False,False,True,False,False,False,False,True,False
77,0.578947,0.079958,0.774684,0.066135,0.789333,0.469758,0.870130,0.193763,0.327445,0.048419,...,False,False,False,True,False,False,True,False,False,False


### Suggestions

-  Filter out (i.e. remove) any irrelevant columns (e.g. names, IDs, etc.)
-  Several columns contain missing values (NaNs). Find out how large a percentage each column is missing. Perhaps some of them lack so much data that you should consider removing them?
-  Scale numeric data.
-  For the features you choose to keep, impute the missing values in an appropriate way - or perhaps you find it more appropriate to delete the rows?
-  Several features (e.g., "PrimaryTrigger") are categorical. Use one-hot encoding to turn them into numeric data. Be careful with the feature "Stages" - perhaps one-hot encoding is not the best choice here?
-  If you you choose to remove or replace outliers, do this now. If you choose to keep, move on.
-  Create a correlation matrix and discuss - based on this, you might want to drop certain columns.
-  Consider whether some features should be transformed (e.g. using log, square root etc.) and do this if found relevant.
-  There are only about 80 rows in the data set. Discuss consequences of this in terms of machine learning - as well as potential solutions. 
-  Think about whether there are other steps you find appropriate at this point. If not, declare your data set clean.